### 3. 使用langchain完成同样的功能
- 首先安装langchain工具
```shell
pip install langchain
```
- 从langchain.chat_models导入OpenAI的对话模型ChatOpenAI。 除去OpenAI以外，langchain.chat_models还集成了其他对话模型，更多细节可以查看Langchain官方文档。

In [14]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
import httpx
from langchain_openai import ChatOpenAI
chat = ChatOpenAI(temperature=0.0,api_key=os.environ['OPENAI_API_KEY'],base_url="https://api-inference.modelscope.cn/v1/", http_client=httpx.Client(verify=False),model="deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B")

In [15]:
chat

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000001F711C4C230>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001F711C4ECF0>, root_client=<openai.OpenAI object at 0x000001F711C325D0>, root_async_client=<openai.AsyncOpenAI object at 0x000001F711C4C470>, model_name='deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B', temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'), openai_api_base='https://api-inference.modelscope.cn/v1/', http_client=<httpx.Client object at 0x000001F70E7C9A90>)

In [16]:
# 美式英语 + 平静、尊敬的语调
style = """American English \
in a calm and respectful tone
"""
text = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

In [17]:
template_string

'Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\n'

In [18]:
from langchain.prompts import ChatPromptTemplate
prompt_template = ChatPromptTemplate.from_template(template_string)

In [19]:
prompt_template.messages[0].prompt

PromptTemplate(input_variables=['style', 'text'], input_types={}, partial_variables={}, template='Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\n')

In [20]:
customer_messages = prompt_template.format_messages(style=style,text=text)	

In [21]:
print(customer_messages[0])

content="Translate the text that is delimited by triple backticks into a style that is American English in a calm and respectful tone\n. text: ```\nArrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse,the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!\n```\n" additional_kwargs={} response_metadata={}


In [22]:
customer_response = chat(customer_messages)

In [23]:
print(customer_response.content)

Here’s the translation in a style that is American English and calm and respectful:

```
Arrr, I’m fuming that I used my blender lid and splattered my smoothies all over the kitchen walls! And to make it worse, the warranty doesn’t cover the cost of cleaning up my kitchen. I need your help right now, matey!
```


上面已经接收到海盗的消息，现在需要用海盗风格回复对方

In [24]:
service_reply = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""
service_style_pirate = """\
a polite tone \
that speaks in English Pirate\
"""

In [25]:
service_messages = prompt_template.format_messages(style=service_style_pirate,text=service_reply)

In [26]:
service_response = chat(service_messages)

In [27]:
service_response.content

'"Hey there, mate! No worries about cleaning expenses because of your mistake that you..." See ya!'

此外，LangChain还提供了提示模版用于一些常用场景。比如summarization, Question answering, or connect to sql databases, or connect to different APIs. 通过使用LongChain内置的提示模版，你可以快速建立自己的大模型应用，而不需要花时间去设计和构造提示

### 4.输出解析器
对于给定的评价customer_review, 我们希望提取信息，并按以下格式输出：
```json
{
  "gift": False,
  "delivery_days": 5,
  "price_value": "pretty affordable!"
}
```

In [30]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

In [31]:
review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [32]:
from langchain.prompts import ChatPromptTemplate
prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

input_variables=['text'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: {text}\n'), additional_kwargs={})]


In [33]:
messages = prompt_template.format_messages(text=customer_review)

In [34]:
response = chat(messages)
print(response.content)

```json
{
  "gift": False,
  "delivery_days": 2,
  "price_value": []
}
```


In [35]:
type(response.content)

str

In [36]:
response.content.get('gift')

AttributeError: 'str' object has no attribute 'get'

In [37]:
# 构造提示模板字符串
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

In [38]:
prompt = ChatPromptTemplate.from_template(template=review_template_2)

In [39]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

gift_schema = ResponseSchema(name="gift",
                             description="Was the item purchased\
                             as a gift for someone else? \
                             Answer True if yes,\
                             False if not or unknown.")

delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="How many days\
                                      did it take for the product\
                                      to arrive? If this \
                                      information is not found,\
                                      output -1.")

price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract any\
                                    sentences about the value or \
                                    price, and output them as a \
                                    comma separated Python list.")

response_schemas = [gift_schema, 
                    delivery_days_schema,
                    price_value_schema]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": string  // Was the item purchased                             as a gift for someone else?                              Answer True if yes,                             False if not or unknown.
	"delivery_days": string  // How many days                                      did it take for the product                                      to arrive? If this                                       information is not found,                                      output -1.
	"price_value": string  // Extract any                                    sentences about the value or                                     price, and output them as a                                     comma separated Python list.
}
```


In [40]:
messages = prompt.format_messages(text=customer_review, format_instructions=format_instructions)

In [41]:
print(messages[0].content)

For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the productto arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,and output them as a comma separated Python list.

text: This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```

In [42]:
response = chat(messages)
print(response.content)

```json
{
	"gift": "True",
	"delivery_days": "2",
	"price_value": "It's slightly more expensive than the other leaf blowers, I think my wife liked it so much she was speechless."
}
```


In [43]:
output_dict = output_parser.parse(response.content)
output_dict

{'gift': 'True',
 'delivery_days': '2',
 'price_value': "It's slightly more expensive than the other leaf blowers, I think my wife liked it so much she was speechless."}

In [44]:
type(output_dict)

dict

In [45]:
output_dict.get('delivery_days')

'2'